# Production Code

In [ ]:
import os
import re
import numpy as np
import pandas as pd

source_directory = "../surveys/plain_txt/"

## Extract

### Global Variables

In [ ]:
fee_list = ["tasas formales\/legales \(por ej\. iva, senasa, monotributo, afip…\)",
            "tasas informales", "vi-9:"]
# column headers not including expense column
column_headers = ["bovines", "sheep", "goats", "camelids", "donkeys",
                  "equines", "pigs"]

### Helper Functions

In [ ]:
def find_between(str_1, str_2, contents):
    regex_between = str_1 + r"([\w\W]*)" + str_2
    regex_search = re.findall(regex_between, contents)

    return regex_search

### Parsing Functions

In [ ]:
def parse_interviewee(contents):
    name_regex = r"nombre de la persona[^\n:]*:[\s]*([^\n]*)\n"
    name_search = re.findall(name_regex, contents)
    if len(name_search) == 1:
        name = name_search[0]
        if name == "3":
            name = "N/A"
    else:
        alt_name_regex = r"nombre de la persona[^\n\(]*\(\w* +([^\n]*)\n"
        alt_name_search = re.findall(alt_name_regex, contents)
        if len(alt_name_search) == 1:
            name = alt_name_search[0]
            if name == "3":
                name = "N/A"
        else:
            name = "UNENCOUNTERED FORMAT"

    name = name.strip()

    return name


def parse_section_6_8(section_search, fee_list, column_headers):
    section_6_8_data = []

    if len(section_search) == 1:
        section_string = section_search[0] + "vi-9:"

        for i in range(len(fee_list)-1):
            item_dict = {}
            item_1 = fee_list[i]
            item_2 = fee_list[i+1]

            row_search = find_between(item_1, item_2, section_string)

            item_dict["fee_type"] = item_1

            # if expense row found
            if len(row_search) == 1:
                row_string = row_search[0]
                row_list = row_string.split("\n")
                row_list_len = len(row_list)

                if row_list_len >= 17:
                    count = -2
                    if (row_list[-1 * count] == ""
                       and row_list[-1 * count + 2] != ""):
                        count = -4

                    for j in range(row_list_len):
                        if count > 12:
                            break
                        if count >= 0 and count % 2 == 0:
                            col = column_headers[count//2]
                            entry = row_list[j]
                            entry = entry.strip()
                            item_dict[col] = entry
                        count += 1
                else:
                    for col in column_headers:
                        item_dict[col] = "UNENCOUNTERED FORMAT"
            else:
                for col in column_headers:
                    item_dict[col] = "UNENCOUNTERED FORMAT"

            section_6_8_data.append(item_dict)
    else:
        section_6_8_data = [{}]

    return section_6_8_data

### Process the Whole File

In [ ]:
def process_files(source_directory, fee_list, column_headers):
    dataset = []

    for process_file in os.listdir(source_directory):
        file_path = os.path.join(source_directory, process_file)

        # with statements automatically control the closing of files
        with open(file_path, "r") as file:
            contents = file.read()
            contents = contents.lower()

            section_start = "vi-8:"
            section_end = "vi-9:"
            # extract only the relevant section
            section_search = find_between(section_start, section_end, contents)

            interviewee = parse_interviewee(contents)
            section_6_8_data = parse_section_6_8(section_search, fee_list,
                                                 column_headers)

            data_dict = {}

            # check to make sure files contain the right amount of entries
            if len(section_6_8_data) <= 1:
                print(f"Unencountered Format: {process_file}\n"
                      + f"Length of section_data: {len(section_6_8_data)}\n")

            for row in section_6_8_data:
                data_dict = {}
                data_dict["filename"] = process_file
                data_dict["interviewee"] = interviewee

                for key, value in row.items():
                    data_dict[key] = value

                dataset.append(data_dict)

    # convert list to DataFrame
    raw_df = pd.DataFrame(data=dataset)

    return raw_df

## Transform

### Helper Functions

In [ ]:
def clean_fee_type(dataf):
    replace_dict = {"tasas formales\/legales \(por ej\. iva, senasa, monotributo, afip…\)": "formal_and_legal",
                    "tasas informales": "informal"}
    dataf["fee_type"] = dataf["fee_type"].replace(replace_dict)

    return dataf

### Pipeline Functions

In [ ]:
def start_pipeline(dataf):
    return dataf.copy()


def handle_null_data(dataf):
    # fill missing values with null
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = isnull_sum >= 7
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf


def clean_values(dataf):
    dataf = clean_fee_type(dataf)

    return dataf

### Create Raw and Cleaned DataFrame

In [ ]:
raw_df = process_files(source_directory, fee_list, column_headers)

clean_df = (raw_df
            .pipe(start_pipeline)
            .pipe(clean_values)
            .pipe(handle_null_data))

## Other
### Functions to Check the Implementation

In [ ]:
def get_dropped_df(raw_dataf, clean_dataf):
    raw_indexes = raw_dataf.index
    clean_indexes = clean_dataf.index

    dropped_indexes = raw_indexes[~raw_indexes.isin(clean_indexes)]
    dropped_df = raw_dataf.loc[dropped_indexes].copy()

    return dropped_df

### Create Dropped DataFrame
Create the `dropped_df` and collect the rows which contain no non-null values

In [ ]:
dropped_df = get_dropped_df(raw_df, clean_df)

# get rows that were dropped but do not contain non-null values
null_df = dropped_df[["filename", "fee_type"]]

### Check Dropped Row

In [ ]:
def get_dropped_row(section_search, fee_type, fee_list):
    dropped_row = {}

    if len(section_search) == 1:
        section_string = section_search[0] + "vi-9:"

        dropped_row["fee_type"] = fee_type

        fee_index = fee_list.index(fee_type)
        item_1 = fee_list[fee_index]
        item_2 = fee_list[fee_index + 1]

        row_search = find_between(item_1, item_2, section_string)

        # if expense row found
        if len(row_search) == 1:
            row_string = row_search[0]
            row_list = row_string.split("\n")
            row_list_len = len(row_list)

            if row_list_len >= 17:
                count = -2
                for j in range(row_list_len):
                    if count > 14:
                        break
                    if count >= 0:
                        col = str(count)
                        entry = row_list[j]
                        entry = entry.strip()
                        dropped_row[col] = entry
                    count += 1

            else:
                for j in range(15):
                    dropped_row[str(j)] = "UNENCOUNTERED FORMAT"
        else:
            for j in range(15):
                dropped_row[str(j)] = "UNENCOUNTERED FORMAT"
    else:
        dropped_row = {}

    return dropped_row

### Check Dropped Rows for Parsing Errors

In [ ]:
def check_dropped_rows(source_directory, fee_list, s):
    process_file = s["filename"]
    fee_type = s["fee_type"]

    check_dict = {}

    file_path = os.path.join(source_directory, process_file)

    # with statements automatically control the closing of files
    with open(file_path, "r") as file:
        contents = file.read()
        contents = contents.lower()

        section_start = "vi-8:"
        section_end = "vi-9:"
        section_search = find_between(section_start, section_end, contents)

        dropped_row = get_dropped_row(section_search, fee_type, fee_list)

        # check to make sure files contain the right amount of entries
        if len(dropped_row) <= 1:
            print(f"Unencountered Format: {process_file}\n"
                  + f"Length of data: {len(dropped_row)}\n")

        check_dict["filename"] = process_file

        for key, value in dropped_row.items():
            check_dict[key] = value

    # convert list to Series
    check_series = pd.Series(data=check_dict)

    return check_series

In [ ]:
check_df = null_df.apply(lambda s: check_dropped_rows(source_directory,
                                                      fee_list, s),
                         axis=1)

In [ ]:
def drop_null_check(dataf):
    # fill missing values with null
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = isnull_sum >= 15
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf

In [ ]:
non_null_df = (check_df
               .pipe(start_pipeline)
               .pipe(drop_null_check))

In [ ]:
non_null_df

## Load
### Export to CSV

In [ ]:
if non_null_df.shape[0] == 0:
    datasets_directory = "../datasets/"
    filename = "section_6_8.csv"
    file_path = os.path.join(datasets_directory, filename)

    clean_df.to_csv(file_path, index=False, na_rep="null")

    print(f"Exported to {filename}")
else:
    print("Error: Dropped rows contain non-null values")

***
# Testing Code
### View the Data

In [ ]:
clean_df.head(50)

### Check for Parsing Errors

In [ ]:
cols_to_check = clean_df.columns
cols_to_drop = ["filename", "interviewee", "fee_type", "bovines"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(clean_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Check Dropped Rows
Another way to check is by opening the variable inspector and manually scrolling
through the `df_dropped` DataFrame. 

The code below tells us whether or not there are files that contain no information for this section. If raw file count is the same as clean file count, then no files were dropped.

In [ ]:
print("Raw File Count: {}".format(len(raw_df["filename"].value_counts())))
print("Clean File Count: {}".format(len(clean_df["filename"].value_counts())))

If there are files that are completely dropped, find those files and look at them to check for parsing errors.

In [ ]:
raw_file_names = raw_df["filename"].value_counts().index
clean_file_names = clean_df["filename"].value_counts().index

raw_file_names[~raw_file_names.isin(clean_file_names)]

Check the rows value counts to see if any rows are being dropped that should not be.

In [ ]:
cols_to_check = dropped_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(dropped_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Duplicate Rows

In [ ]:
df_columns = clean_df.columns[clean_df.columns != "filename"]
clean_df[clean_df.loc[:, df_columns].duplicated(keep=False)]

----
## Experimental Code

In [ ]:
process_file = "11 Roberto Claudio Gutierrez.txt"


file_path = os.path.join(source_directory, process_file)

# with statements automatically control the closing of files
with open(file_path, "r") as file:
    contents = file.read()
    contents = contents.lower()

    section_start = "vi-6:"
    section_end = "vi-8:"
    # extract only the relevant section
    section_search = find_between(section_start, section_end, contents)

In [ ]:
section_6_6_data = []

if len(section_search) == 1:
    section_string = section_search[0] + "vi-8:"

    item_dict = {}
    item_1 = "energía"
    item_2 = "calefacción"

    row_search = find_between(item_1, item_2, section_string)

    item_dict["expense"] = item_1

    # if expense row found
    if len(row_search) == 1:
        row_string = row_search[0]
        row_list = row_string.split("\n")
        row_list_len = len(row_list)

        if row_list_len >= 9:
            count = -2
            if (row_list[-1 * count] == ""
               and row_list[-1 * count + 2] != ""):
                count -= 2

            for j in range(row_list_len):
                if count > 4:
                    break
                if count >= 0 and count % 2 == 0:
                    col = column_headers[count//2]
                    entry = row_list[j]
                    entry = entry.strip()
                    item_dict[col] = entry
                count += 1
        else:
            for col in column_headers:
                item_dict[col] = "UNENCOUNTERED FORMAT"
    else:
        for col in column_headers:
            item_dict[col] = "UNENCOUNTERED FORMAT"

        section_6_6_data.append(item_dict)

In [ ]:
row_list

In [ ]:
item_dict